In [25]:
#SETUP always run this
import math
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

sectors = json.load(open("sectors.json",'r'))
closingPrices = pd.read_csv("latestClose.csv")
closingPrices = closingPrices.drop(columns=["Date"])
closingPrices.tail()

allSectors = list(sectors.keys())
allIndustries = []
for sector in allSectors:
    allIndustries.extend(list(sectors[sector].keys()))
allIndustries = list(set(allIndustries))
allTickers = []
for sector in allSectors:
    for industry in sectors[sector]:
        allTickers.extend(sectors[sector][industry])
allTickers = list(set(allTickers))

industries = {}
for sector in allSectors:
    industries[sector] = list(sectors[sector].keys())

print("sectors:", allSectors)
print(len(industries))
print(len(allTickers))

sectors: ['Consumer Discretionary', 'Technology', 'Health Care', 'Financial', 'Real Estate', 'Communication Services', 'Consumer Staples', 'Industrial', 'Energy', 'Materials', 'Utilities']
11
3390


In [ ]:
def getAllData(tickers):
    anchorData = yf.download(tickers, start = "2021-01-01")
    anchorData.head()
    closingPrices = anchorData["Close"]
    closingPrices.to_csv("latestClose.csv")
    closingPrices = pd.read_csv("latestClose.csv")
    return closingPrices

def getSearchRange(start, end):
    return closingPrices[closingPrices['Date'].between(start, end)]

closingPrices = getAllData(tickers)


In [ ]:
industries

In [ ]:
movingAverages = {}
for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        count = 0
        total = 0
        for ticker in sectors[sector][industry]:
            movingAverages[(ticker, 20)] = closingPrices[ticker][-20::].mean()
            movingAverages[(ticker, 50)] = closingPrices[ticker][-50::].mean()
            movingAverages[(ticker, 200)] = closingPrices[ticker][-200::].mean()

In [ ]:
print(len(movingAverages), movingAverages)

In [ ]:
MAs = [20,50,200]
counts = {}
for sector in allSectors:
    sectorCount = {}
    sectorTotal = 0
    print(sector)
    for industry in sectors[sector]:
        count = {}
        total = 0
        for ticker in sectors[sector][industry]:
            sectorTotal +=1
            total +=1
            for ma in MAs:
                if closingPrices[ticker].iloc[-1] > movingAverages[(ticker, ma)]:
                    count[ma] = count.get(ma,0)+1
                    sectorCount[ma] = sectorCount.get(ma,0)+1

        for i in count:
            print(industry, i, count[i], total)

        percents = {i : count[i]/total for i in count}
        counts[industry] = percents
    sectorPercent = {i : sectorCount[i]/sectorTotal for i in sectorCount}
    counts[sector] = sectorPercent

In [ ]:
percents = {}
for sector in allSectors:
    percents[sector] = {}
    percents[sector]["Sector"] = counts[sector]
    for industry in sectors[sector]:
        percents[sector][industry] = counts[industry]
percents

In [ ]:
for sector in allSectors:
    print (sector)

In [ ]:
#Print Percents to a CSV. Organized by Sector, then Industry
with open("percentages.csv", "w") as f:
    f.write("Moving Average Breadth Dashboard,20,50,200,\n")
    f.write("\n")
    for sector in allSectors:
        f.write(sector.upper() + ",")
        for ma in MAs:
            output = round(percents[sector]["Sector"][ma]*100,2)
            f.write(str(output) + ",")
        f.write("\n")
        for industry in sectors[sector]:
            f.write(industry + ",")
            for ma in MAs:
                if ma in percents[sector][industry]:
                    output = round(percents[sector][industry][ma]*100,2)
                    f.write(str(output) + ",")
                else:
                    f.write("0,")
            f.write("\n")
        f.write("\n")
        
    

In [29]:
percents["Technology"]["Renewable Energy Equipment"]

{20: 0.1111111111111111, 50: 0.1111111111111111, 200: 0.1111111111111111}

In [30]:
sectors["Technology"]["Renewable Energy Equipment"]

['EVGO',
 'FTCI',
 'MAXN',
 'TPIC',
 'ALLG',
 'BLDP',
 'FCEL',
 'ARRY',
 'CSIQ',
 'JKS',
 'NOVA',
 'RUN',
 'SHLS',
 'SPWR',
 'ENPH',
 'FSLR',
 'PLUG',
 'SEDG']